In [1]:
import os
import time
from datetime import datetime, timedelta

In [2]:
os.add_dll_directory('C:\\Program Files\\IBM\\SQLLIB\\BIN')
# conectar a la base de datos IBM Db2
import ibm_db

In [3]:
from queries import get_academic_history, get_socioeconomico, tasa_reprobacion, dificultad_materia
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
def format_time(seconds):
    """Convierte segundos a formato HH:MM:SS"""
    return str(timedelta(seconds=int(seconds)))

def print_progress_bar(current, total, bar_length=30):
    """Imprime una barra de progreso"""
    percent = float(current) * 100 / total
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    return f'[{bar}] {percent:.1f}% ({current}/{total})'

In [5]:
print("="*80)
print("INICIO DE DESCARGA DE DATOS ACADÉMICOS")
print("="*80)
print(f"Fecha y hora de inicio: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

INICIO DE DESCARGA DE DATOS ACADÉMICOS
Fecha y hora de inicio: 2025-09-30 20:01:32


In [6]:
# Tiempo de inicio total
start_time_total = time.time()

print("\n📡 Conectando con la base de datos...")
start_time_conn = time.time()


📡 Conectando con la base de datos...


In [7]:
conn_str = (
    'DATABASE=' + os.getenv('DATABASE') + 
    ';HOSTNAME=' + os.getenv('HOSTNAME') + 
    ';PORT=' + os.getenv('PORT') + 
    ';PROTOCOL=TCPIP;UID=' + os.getenv('USERNAME_DB') + 
    ';PWD=' + os.getenv('PASSWORD_DB') + ';'
)

print(repr(conn_str))
conn = ibm_db.connect(conn_str, '', '')

'DATABASE=SAAC;HOSTNAME=192.168.254.53;PORT=50000;PROTOCOL=TCPIP;UID=espol;PWD=espol123;'


In [8]:
end_time_conn = time.time()
connection_time = end_time_conn - start_time_conn

if conn:
    print(f"✅ Conexión exitosa - Tiempo: {connection_time:.2f} segundos")
else:
    print("❌ Error al conectar")
    exit(1)

✅ Conexión exitosa - Tiempo: 84.68 segundos


In [9]:
# Cargar datos de materias
print("\n📁 Cargando lista de materias...")
start_time_load = time.time()
df_materias = pd.read_csv("./data/materias/dificultad_materia.csv")
end_time_load = time.time()
load_time = end_time_load - start_time_load

print(f"✅ Materias cargadas - Tiempo: {load_time:.2f} segundos")
print(f"📊 Total de materias en archivo: {len(df_materias)}")


📁 Cargando lista de materias...
✅ Materias cargadas - Tiempo: 0.01 segundos
📊 Total de materias en archivo: 1926


In [10]:
# # Filtrar materias a procesar
# list_not_grades = ["PORTAFOLIO"]
# materias_to_process = []

# for i in df_materias.iterrows():
#     if i[1]["MATERIA_x"] in list_not_grades or "MATERIA INTEGRADORA" in i[1]["MATERIA_x"]:
#         continue
#     materias_to_process.append(i[1])
materias_to_process = df_materias[df_materias["CLASIFMATERIA"] == "G"]

In [11]:
total_materias = len(materias_to_process)
print(f"🎯 Materias a procesar: {total_materias}")

# Parámetros de configuración
year = "2020"
semester = "1S"
print(f"📅 Período: {year} - {semester}")

🎯 Materias a procesar: 1892
📅 Período: 2020 - 1S


In [12]:
print("\n" + "="*80)
print("INICIANDO PROCESAMIENTO DE MATERIAS")
print("="*80)

# Variables para estadísticas
total_students_processed = 0
failed_materias = []
processing_times = []
df_complete = pd.DataFrame()


INICIANDO PROCESAMIENTO DE MATERIAS


In [13]:
# Procesar cada materia
# for counter, materia_data in enumerate(materias_to_process, 1):
for counter, materia_data in enumerate(materias_to_process["CODIGOMATERIA"], 1):
    print(counter, materia_data)
    print(f"\n{'='*60}")
    print(f"PROCESANDO MATERIA {counter}/{total_materias}")
    print(f"{'='*60}")
    
    cod_materia = materia_data.strip()
    # cod_materia = materia_data["CODIGOMATERIA"].strip()
    # materia_name = materia_data["MATERIA_x"]
    
    print(f"📚 Código: {cod_materia}")
    # print(f"📖 Nombre: {materia_name}")
    print(f"⏱️  {print_progress_bar(counter-1, total_materias)}")
    
    # Tiempo de inicio para esta materia
    start_time_materia = time.time()
    
    try:
        # Query 1: Historial académico
        print(f"\n🔍 Ejecutando query de historial académico...")
        start_time_query1 = time.time()
        
        query = get_academic_history(cod_materia, year, semester)
        stmt_select = ibm_db.exec_immediate(conn, query)
        
        data_list = []
        result = ibm_db.fetch_assoc(stmt_select)
        while result:
            result["COD_MATERIA_ACAD_MO"] = result["COD_MATERIA_ACAD_MO"].strip()
            data_list.append(result)
            result = ibm_db.fetch_assoc(stmt_select)
        
        df = pd.DataFrame(data_list)
        end_time_query1 = time.time()
        query1_time = end_time_query1 - start_time_query1
        
        num_students = len(df)
        print(f"✅ Query 1 completada - Estudiantes encontrados: {num_students} - Tiempo: {query1_time:.2f}s")
        
        if num_students == 0:
            print(f"⚠️  No hay estudiantes para esta materia, saltando...")
            continue
        
        # # Query 2: Datos socioeconómicos
        # print(f"🔍 Ejecutando query socioeconómico...")
        # start_time_query2 = time.time()

        # query = get_socioeconomico(df["COD_ESTUDIANTE"].to_list())
        # stmt_select = ibm_db.exec_immediate(conn, query)

        # data_list = []
        # result = ibm_db.fetch_assoc(stmt_select)
        # while result:
        #     data_list.append(result)
        #     result = ibm_db.fetch_assoc(stmt_select)

        # df_socio = pd.DataFrame(data_list)
        # end_time_query2 = time.time()
        # query2_time = end_time_query2 - start_time_query2

        # socio_records = len(df_socio)
        socio_records = len(df)
        # print(f"✅ Query 2 completada - Registros socioeconómicos: {socio_records} - Tiempo: {query2_time:.2f}s")

        # if df_socio.empty:
        #     print(f"⚠️ No se encontraron datos socioeconómicos de {len(df["COD_ESTUDIANTE"])} estudiante/s. Se crearán columnas vacías.")
        #     df_socio = pd.DataFrame(columns=['CODESTUDIANTE']) # Asegura que la columna exista.


        # # Merge y guardado
        # print(f"🔗 Realizando merge y guardando archivo...")
        # start_time_merge = time.time()

        # df["COD_ESTUDIANTE"] = df["COD_ESTUDIANTE"].astype(str).str.strip()
        # df_socio["CODESTUDIANTE"] = df_socio["CODESTUDIANTE"].astype(str).str.strip()

        # df_merge = pd.merge(df, df_socio, left_on='COD_ESTUDIANTE', right_on="CODESTUDIANTE", how='left')

        # if "CODESTUDIANTE" in df_merge.columns:
        #     df_merge.drop("CODESTUDIANTE", axis=1, inplace=True)
            
        # df_complete = pd.concat([df_complete, df_merge], ignore_index=True)
        df_complete = pd.concat([df_complete, df], ignore_index=True)

        end_time_merge = time.time()
        # merge_time = end_time_merge - start_time_merge

        # Tiempo total para esta materia
        end_time_materia = time.time()
        materia_time = end_time_materia - start_time_materia
        processing_times.append(materia_time)
        
        
        # print(f"💾 Merge y guardado - Tiempo: {merge_time:.2f}s")
        # print(f"📊 Registros finales: {len(df_merge)}")
        
        # Estadísticas de la materia
        total_students_processed += num_students
        coverage = (socio_records / num_students * 100) if num_students > 0 else 0
        
        print(f"\n📈 RESUMEN MATERIA:")
        print(f"   • Estudiantes académicos: {num_students}")
        # print(f"   • Registros socioeconómicos: {socio_records}")
        print(f"   • Cobertura socioeconómica: {coverage:.1f}%")
        print(f"   • Query 1: {query1_time:.2f}s")
        # print(f"   • Query 2: {query2_time:.2f}s")
        # print(f"   • Merge/Guardado: {merge_time:.2f}s")
        print(f"   • Tiempo total materia: {format_time(materia_time)}")
        
        # Estimación de tiempo restante
        if len(processing_times) >= 3:  # Solo después de procesar al menos 3 materias
            avg_time_per_materia = sum(processing_times) / len(processing_times)
            remaining_materias = total_materias - counter
            estimated_remaining_time = avg_time_per_materia * remaining_materias
            
            print(f"⏰ Tiempo promedio por materia: {format_time(avg_time_per_materia)}")
            print(f"⏳ Tiempo estimado restante: {format_time(estimated_remaining_time)}")
            
            estimated_finish = datetime.now() + timedelta(seconds=estimated_remaining_time)
            print(f"🏁 Finalización estimada: {estimated_finish.strftime('%H:%M:%S')}")
        
        # if counter == 2:
        #     break
    except Exception as e:
        print(f"❌ ERROR procesando {cod_materia}: {str(e)}")
        failed_materias.append({
            'codigo': cod_materia,
            # 'nombre': materia_name,
            'error': str(e)
        })

1 ACUG1035

PROCESANDO MATERIA 1/1892
📚 Código: ACUG1035
⏱️  [------------------------------] 0.0% (0/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 15 - Tiempo: 20.55s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 15
   • Cobertura socioeconómica: 100.0%
   • Query 1: 20.55s
   • Tiempo total materia: 0:00:20
2 ACUG1036

PROCESANDO MATERIA 2/1892
📚 Código: ACUG1036
⏱️  [------------------------------] 0.1% (1/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 22 - Tiempo: 20.88s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 22
   • Cobertura socioeconómica: 100.0%
   • Query 1: 20.88s
   • Tiempo total materia: 0:00:20
3 ACUG1039

PROCESANDO MATERIA 3/1892
📚 Código: ACUG1039
⏱️  [------------------------------] 0.1% (2/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 5 - Tiempo: 19.37s

📈 RESUMEN MATERIA:
   • Estudiantes académ

C:\Users\saraujo\AppData\Local\Temp\ipykernel_45512\3559937080.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df], ignore_index=True)


✅ Query 1 completada - Estudiantes encontrados: 27 - Tiempo: 19.48s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 27
   • Cobertura socioeconómica: 100.0%
   • Query 1: 19.48s
   • Tiempo total materia: 0:00:19
⏰ Tiempo promedio por materia: 0:00:20
⏳ Tiempo estimado restante: 10:34:26
🏁 Finalización estimada: 06:50:57
45 ADSG1022

PROCESANDO MATERIA 45/1892
📚 Código: ADSG1022
⏱️  [------------------------------] 2.3% (44/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 33 - Tiempo: 19.69s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 33
   • Cobertura socioeconómica: 100.0%
   • Query 1: 19.69s
   • Tiempo total materia: 0:00:19
⏰ Tiempo promedio por materia: 0:00:20
⏳ Tiempo estimado restante: 10:33:21
🏁 Finalización estimada: 06:50:12
46 ADSG1023

PROCESANDO MATERIA 46/1892
📚 Código: ADSG1023
⏱️  [------------------------------] 2.4% (45/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes enc

C:\Users\saraujo\AppData\Local\Temp\ipykernel_45512\3559937080.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df], ignore_index=True)


✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 5.16s
⚠️  No hay estudiantes para esta materia, saltando...
91 ARQG2023

PROCESANDO MATERIA 91/1892
📚 Código: ARQG2023
⏱️  [█-----------------------------] 4.8% (90/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 3 - Tiempo: 13.14s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 3
   • Cobertura socioeconómica: 100.0%
   • Query 1: 13.14s
   • Tiempo total materia: 0:00:13
⏰ Tiempo promedio por materia: 0:00:18
⏳ Tiempo estimado restante: 9:06:22
🏁 Finalización estimada: 05:33:15
92 ARQG2025

PROCESANDO MATERIA 92/1892
📚 Código: ARQG2025
⏱️  [█-----------------------------] 4.8% (91/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 5 - Tiempo: 13.51s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 5
   • Cobertura socioeconómica: 100.0%
   • Query 1: 13.51s
   • Tiempo total materia: 0:00:13
⏰ Tiempo promedio por materia: 0:

C:\Users\saraujo\AppData\Local\Temp\ipykernel_45512\3559937080.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df], ignore_index=True)


✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 5.08s
⚠️  No hay estudiantes para esta materia, saltando...
331 EDCOM00323

PROCESANDO MATERIA 331/1892
📚 Código: EDCOM00323
⏱️  [█████-------------------------] 17.4% (330/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 4.91s
⚠️  No hay estudiantes para esta materia, saltando...
332 EDCOM00380

PROCESANDO MATERIA 332/1892
📚 Código: EDCOM00380
⏱️  [█████-------------------------] 17.5% (331/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 4.53s
⚠️  No hay estudiantes para esta materia, saltando...
333 EDCOM00398

PROCESANDO MATERIA 333/1892
📚 Código: EDCOM00398
⏱️  [█████-------------------------] 17.5% (332/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 5.11s
⚠️  No hay estudiantes para esta materia, saltando...
334 EDCOM00406

PROCESAND

C:\Users\saraujo\AppData\Local\Temp\ipykernel_45512\3559937080.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df], ignore_index=True)


✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 5.20s
⚠️  No hay estudiantes para esta materia, saltando...
1579 NAVG1038

PROCESANDO MATERIA 1579/1892
📚 Código: NAVG1038
⏱️  [█████████████████████████-----] 83.4% (1578/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 0 - Tiempo: 4.80s
⚠️  No hay estudiantes para esta materia, saltando...
1580 NAVG1039

PROCESANDO MATERIA 1580/1892
📚 Código: NAVG1039
⏱️  [█████████████████████████-----] 83.5% (1579/1892)

🔍 Ejecutando query de historial académico...
✅ Query 1 completada - Estudiantes encontrados: 30 - Tiempo: 13.70s

📈 RESUMEN MATERIA:
   • Estudiantes académicos: 30
   • Cobertura socioeconómica: 100.0%
   • Query 1: 13.70s
   • Tiempo total materia: 0:00:13
⏰ Tiempo promedio por materia: 0:00:15
⏳ Tiempo estimado restante: 1:20:44
🏁 Finalización estimada: 00:56:27
1581 NAVG1040

PROCESANDO MATERIA 1581/1892
📚 Código: NAVG1040
⏱️  [█████████████████████████-----] 83.5% (15

In [14]:
df_complete.shape

(35871, 25)

In [15]:
filename = f"./data/materias/all_{year}_{semester}.csv"
df_complete.to_csv(filename, index=False)
print(f"✅ Archivo guardado: {filename}")

✅ Archivo guardado: ./data/materias/all_2020_1S.csv


In [16]:
# Cerrar conexión
print(f"\n🔌 Cerrando conexión a la base de datos...")
ibm_db.close(conn)

# Tiempo total
end_time_total = time.time()
total_time = end_time_total - start_time_total



🔌 Cerrando conexión a la base de datos...


In [17]:
print("\n" + "="*80)
print("RESUMEN FINAL DEL PROCESAMIENTO")
print("="*80)
print(f"🏁 Fecha y hora de finalización: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"⏱️  Tiempo total de ejecución: {format_time(total_time)}")
print(f"📊 Materias procesadas exitosamente: {total_materias - len(failed_materias)}/{total_materias}")
print(f"👥 Total estudiantes procesados: {total_students_processed:,}")



RESUMEN FINAL DEL PROCESAMIENTO
🏁 Fecha y hora de finalización: 2025-10-01 00:18:34
⏱️  Tiempo total de ejecución: 4:17:01
📊 Materias procesadas exitosamente: 1892/1892
👥 Total estudiantes procesados: 35,871


In [18]:
if processing_times:
    avg_time = sum(processing_times) / len(processing_times)
    min_time = min(processing_times)
    max_time = max(processing_times)
    
    print(f"\n📈 ESTADÍSTICAS DE TIEMPO:")
    print(f"   • Tiempo promedio por materia: {format_time(avg_time)}")
    print(f"   • Tiempo mínimo: {format_time(min_time)}")
    print(f"   • Tiempo máximo: {format_time(max_time)}")
    print(f"   • Velocidad promedio: {total_students_processed/total_time:.1f} estudiantes/segundo")


📈 ESTADÍSTICAS DE TIEMPO:
   • Tiempo promedio por materia: 0:00:15
   • Tiempo mínimo: 0:00:12
   • Tiempo máximo: 0:01:06
   • Velocidad promedio: 2.3 estudiantes/segundo


In [19]:
if failed_materias:
    print(f"\n❌ MATERIAS CON ERRORES ({len(failed_materias)}):")
    for failed in failed_materias:
        print(f"   • {failed['codigo']} - {failed['nombre'][:50]}")
        print(f"     Error: {failed['error']}")

❌ MATERIAS CON ERRORES (15):
   • ACUG1045 - PRODUCCIÓN ACUÍCOLA II
     Error: 'CODESTUDIANTE'
   • ALIG1033 - DISEÑO DE PLANTAS ALIMENTARIAS
     Error: 'CODESTUDIANTE'
   • CCPG1058 - SISTEMAS DE INFORMACIÓN APLICADOS A LOGÍSTICA
     Error: 'CODESTUDIANTE'
   • ESTG1056 - DATOS NO ESTRUCTURADOS
     Error: 'CODESTUDIANTE'
   • GEOG1027 - DISEÑO DE CAMPO GEOLÓGICO
     Error: 'CODESTUDIANTE'
   • GEOG1037 - INGENIERÍA GEOLÓGICA
     Error: 'CODESTUDIANTE'
   • INDG1061 - HSEQ EN LA INDUSTRIA HIDROCARBURÍFERA
     Error: 'CODESTUDIANTE'
   • MING1031 - PLANEAMIENTO Y DISEÑO MINERO
     Error: 'CODESTUDIANTE'
   • MTRG1027 - DISEÑO DE MATERIALES COMPUESTOS
     Error: 'CODESTUDIANTE'
   • MTRG1041 - NANOTECNOLOGÍA Y NANOMATERIALES
     Error: 'CODESTUDIANTE'
   • PETG1022 - INGENIERÍA DE PRODUCCIÓN II
     Error: 'CODESTUDIANTE'
   • PETG1023 - INGENIERÍA DE YACIMIENTOS I
     Error: 'CODESTUDIANTE'
   • PETG1024 - INGENIERÍA DE YACIMIENTOS II
     Error: 'CODESTUDIANTE'
   • PETG1028 - SIMULACIÓN DE RESERVORIOS
     Error: 'CODESTUDIANTE'
   • PETG1034 - EVALUACIÓN DE FORMACIONES II
     Error: 'CODESTUDIANTE'

In [20]:

print(f"\n✅ Procesamiento completado!")
print("="*80)


✅ Procesamiento completado!
